In [1]:
import xarray as xr
import pandas as pd
import numpy as np
from datetime import datetime as dt
from metpy.units import units
from metpy import calc as mpcalc
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import requests
from pathlib import Path
import datetime


import geocat.viz as gv
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.animation as animation


In [2]:
HOME = str(Path.home())
credFile = open(HOME+'/.rdarc','r')
userId, pw = credFile.read().split()

In [3]:
year =  2018
month = 7
day = 29
hour = 0
ntimes = 4*60


min_lon = -140. + 360.
max_lon =  -5.  + 360.
delta_lon = 20.
min_lat = 0.
max_lat =  30.

# encode the date 
date = datetime.datetime(year,month,day,hour)


In [4]:
era5_base ='https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/'

In [5]:
date.strftime('%Y%m')

'201807'

In [6]:
varId  = '138'
varNam = 'vo'

In [7]:
session = requests.Session()
session.auth = (userId, pw)

            

In [8]:
def animate(i,dateIn,min_lon,max_lon,min_lat,max_lat,Cmin,Cmax):
    central_lon = .5*(min_lon + max_lon)
    delta_lon = 20.
    date = dateIn + pd.Timedelta(days=i*.25)
    print(date)
    fnam = date.strftime('%Y%m') + '/e5.oper.an.pl.128_' + varId + '_' + varNam + \
    '.ll025sc.' + date.strftime('%Y%m%d')+'00_' + date.strftime('%Y%m%d')+'23.nc'   
    ds_url = era5_base + fnam
    vorf = xr.backends.PydapDataStore.open(ds_url, session=session)
    ds = xr.open_dataset(vorf)
    vo = ds.VO.sel(time=date,level=850, latitude=slice(max_lat, min_lat), longitude= slice(min_lon, max_lon) )*1.e5
    cf = vo.plot.contourf(ax=ax,
                          transform=projection,
                          levels=np.arange(-20, 20, 2),
                          cmap='bwr',extend='both',
                          add_colorbar=False)

    
    
    plt.title(date, y=1.2)
    

In [9]:
fnam = date.strftime('%Y%m') + '/e5.oper.an.pl.128_' + varId + '_' + varNam + \
'.ll025sc.' + date.strftime('%Y%m%d')+'00_' + date.strftime('%Y%m%d')+'23.nc'   
ds_url = era5_base + fnam

vorf = xr.backends.PydapDataStore.open(ds_url, session=session)
ds = xr.open_dataset(vorf)
vo = ds.VO.sel(time=date,level=850, latitude=slice(max_lat, min_lat), longitude= slice(min_lon, max_lon) )*1.e5

In [10]:
vo

<xarray.DataArray 'VO' (latitude: 121, longitude: 541)>
array([[-3.9012871e+00, -3.8565836e+00, -3.6986313e+00, ...,
        -3.3171616e+00, -3.4423313e+00, -1.2786826e+00],
       [-4.0026150e+00, -3.9072475e+00, -3.5764418e+00, ...,
         4.0514860e-01,  6.4952767e-01,  2.2439518e+00],
       [-4.1873894e+00, -4.2559347e+00, -4.1158638e+00, ...,
         2.5300541e+00,  2.2335443e-01, -1.4813384e+00],
       ...,
       [-3.7567225e-01, -4.5911875e-01, -1.8195716e-01, ...,
         1.7269048e-01,  3.1276140e-01,  4.2302999e-01],
       [-2.0877925e-01, -2.8924552e-01, -1.3427344e-01, ...,
         8.3283521e-02,  2.7103814e-01,  2.5911722e-01],
       [-4.1886233e-02, -6.1234459e-03, -2.0281878e-01, ...,
         2.8172508e-03,  1.5778932e-01,  2.5017652e-01]], dtype=float32)
Coordinates:
  * latitude   (latitude) float64 30.0 29.75 29.5 29.25 ... 0.75 0.5 0.25 0.0
    level      float64 850.0
  * longitude  (longitude) float64 220.0 220.2 220.5 220.8 ... 354.5 354.8 355.0
    time       datetime64[ns] 2018-07-29

In [11]:

# set the min and max values to shade in the IR brightness temperature field
Cmin=  -10
Cmax = 10
Cint = 1

print(vo.min().values, vo.max().values)

-28.32429 54.412918


In [ ]:
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.ioff()


# Generate figure and set its size (width, height) in inches
fig = plt.figure(figsize=(10,8))

# Generate axes, using Cartopy, drawing coastlines, and adding features
projection = ccrs.PlateCarree()
ax = plt.axes(projection=projection)
ax.coastlines(linewidths=0.5)
ax.add_feature(cfeature.LAND, facecolor='lightgray')

cf = vo.plot.contourf(ax=ax,
                          transform=projection,
                          levels=np.arange(-10, 10, 1),
                          cmap='bwr',extend='both',
                          add_colorbar=False)
# Add colorbar
cbar = plt.colorbar(cf,
                    shrink=0.8,
                    drawedges=True,
                    ticks=np.arange(-20,20,4),
                    extendrect=True,
                    extendfrac='auto', orientation='horizontal')

lon1tick=280
lon2tick=340

gl = ax.gridlines(draw_labels=True)
#ax.coastlines('50m', linewidth=0.8,color='gray')
gl.top_labels = gl_right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER



#---------------------------------------------------------------------------------------------------
animArgs = (date,min_lon,max_lon,min_lat,max_lat,Cmin,Cmax)
# runs the animation initiated with the frame from init and progressed with the animate function
anim = animation.FuncAnimation(fig,animate,frames=ntimes, interval=200, fargs=animArgs)
anim
anim.save('test.mp4')


2018-07-29 00:00:00
2018-07-29 00:00:00
2018-07-29 06:00:00
2018-07-29 12:00:00
2018-07-29 18:00:00
2018-07-30 00:00:00
2018-07-30 06:00:00
2018-07-30 12:00:00
2018-07-30 18:00:00
2018-07-31 00:00:00
2018-07-31 06:00:00
2018-07-31 12:00:00
2018-07-31 18:00:00
2018-08-01 00:00:00
2018-08-01 06:00:00
2018-08-01 12:00:00
2018-08-01 18:00:00
2018-08-02 00:00:00
2018-08-02 06:00:00
2018-08-02 12:00:00
2018-08-02 18:00:00
2018-08-03 00:00:00
2018-08-03 06:00:00
2018-08-03 12:00:00
2018-08-03 18:00:00
2018-08-04 00:00:00
2018-08-04 06:00:00
2018-08-04 12:00:00
2018-08-04 18:00:00
2018-08-05 00:00:00
2018-08-05 06:00:00
2018-08-05 12:00:00
2018-08-05 18:00:00
2018-08-06 00:00:00
2018-08-06 06:00:00
2018-08-06 12:00:00
2018-08-06 18:00:00
2018-08-07 00:00:00
2018-08-07 06:00:00
2018-08-07 12:00:00
2018-08-07 18:00:00
2018-08-08 00:00:00
2018-08-08 06:00:00
2018-08-08 12:00:00
2018-08-08 18:00:00
2018-08-09 00:00:00
2018-08-09 06:00:00
2018-08-09 12:00:00
2018-08-09 18:00:00
2018-08-10 00:00:00


In [ ]:
anim.save('test.mp4')
